In [ ]:
!pip install sentence-transformers
!pip install --upgrade nltk

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('all')

In [5]:
import csv
import sys
import string
import re
import os
import pandas as pd
import numpy as np

from nltk.tokenize import word_tokenize
from nltk.translate import bleu
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from nltk.translate.nist_score import sentence_nist

from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn import neural_network
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from sentence_transformers import SentenceTransformer

In [6]:
def process(words):
  words = words.replace("<br />", " ").rstrip()
  temp = str.maketrans(string.punctuation, ' '*len(string.punctuation))
  words = words.translate(temp)
  # words = words.lower()
  words = words.split()
  words = [word for word in words if word not in stopwords]
  processed_words = ' '.join(words)
  return processed_words

def strip_punc(s):
  return s.translate(str.maketrans('', '', string.punctuation))

def set_stopwords():
  stopwords = open("/content/drive/My Drive/translations/stopwords.en.txt", "r", encoding="utf8")
  words = stopwords.read().split("\n")
  stopwords.close()
  return words

In [7]:
def build_matrix(cols,data):
  n = data.shape[0]
  X = np.zeros((n,1))
  vec_cols = []
  for col in cols:
    if col in vec_cols:
      array = np.asarray(list(data[col]))
      X = np.c_[ X, array ] 
    else:
      array = np.asarray(data[col])
      array = array.reshape(array.shape[0],-1)
      X = np.c_[ X, array ] 
  return X[:,1:]

In [8]:
def cosine_similarity(A, B):
  return (np.dot(A,B))/(np.linalg.norm(A)*np.linalg.norm(B))

In [9]:
def load_data(directory):

  ############ Load training data

  sources = []
  references = []
  candidates = []
  scores = []
  labels = []
  targets = []

  with open(f"{directory}/train.txt", encoding="utf-8") as f:
    reader = f.readlines()
    for count,row in enumerate(reader):
      if count % 6 == 0:
        sources.append(row.strip('\n'))
      elif count % 6 == 1:
        references.append( strip_punc(row.strip('\n')) )  
      elif count % 6 == 2:
        candidates.append( strip_punc(row.strip('\n')) ) 
      elif count % 6 == 3:
        scores.append(row.strip('\n'))
      elif count % 6 == 4:
        labels.append(row.strip('\n'))
        if (row.strip('\n') == 'H'):
          targets.append(1)
        else:
          targets.append(0)

  dict = {
      'source':sources,
      'reference':references,
      'candidate':candidates,
      'bleu_uni':scores,
      'label':labels,
      'target':targets
          }
  train_data = pd.DataFrame(dict)

  ############ Load test data

  sources = []
  references = []
  candidates = []
  scores = []
  labels = []
  targets = []

  with open(f"{directory}/test.txt", encoding="utf-8") as f:
  
    reader = f.readlines()
    for count,row in enumerate(reader):
      if count % 6 == 0:
        sources.append(row.strip('\n'))
      elif count % 6 == 1:
        references.append( strip_punc(row.strip('\n')) )
      elif count % 6 == 2:
        candidates.append( strip_punc(row.strip('\n')) )
      elif count % 6 == 3:
        scores.append(row.strip('\n'))
      elif count % 6 == 4:
        labels.append(row.strip('\n'))
        if (row.strip('\n') == 'H'):
          targets.append(1)
        else:
          targets.append(0)
    
  dict = {
      'source':sources,
      'reference':references,
      'candidate':candidates,
      'bleu_uni':scores,
      'label':labels,
      'target':targets
          }
  test_data = pd.DataFrame(dict)

  return train_data, test_data

In [ ]:

############################################################
############ Begin execution

sbert_model = SentenceTransformer('distiluse-base-multilingual-cased-v2')

In [11]:
############ Load data from files

stopwords = set_stopwords()
directory = '/content/drive/My Drive/translations'
train_data, test_data = load_data(directory)

############ Form custom embedding features for train, test

dfs = [train_data, test_data]

for df in dfs:

  cos_s_r = []
  cos_s_c = []
  cos_c_r = []

  for i in range(len(df.index)):

    row = df.loc[i]

    text = row['source']
    sour_vec = sbert_model.encode([text])[0]

    text = process(row['reference'])
    ref_vec = sbert_model.encode([text])[0]

    text = process(row['candidate'])
    can_vec = sbert_model.encode([text])[0]

    cos_s_r.append( cosine_similarity(sour_vec, ref_vec) )
    cos_s_c.append( cosine_similarity(sour_vec, can_vec) )
    cos_c_r.append( cosine_similarity(can_vec, ref_vec) )

  df['cos_s_r'] = cos_s_r
  df['cos_s_c'] = cos_s_c
  df['cos_c_r'] = cos_c_r

In [13]:
############ Compute additional MT Scores for train, test 

smoother = SmoothingFunction().method4

for df in dfs:

  bleu_sm = []
  met = []
  nist = []

  for i in range(len(df.index)):

    row = df.loc[i]

    ref_tokens = word_tokenize(row['reference'])
    can_tokens = word_tokenize(row['candidate'])
    bleu_sm.append( sentence_bleu([ref_tokens], can_tokens, smoothing_function=smoother) )
    met.append( nltk.translate.meteor_score.meteor_score([row['reference']], row['candidate']) )
    nist.append( sentence_nist([ref_tokens], can_tokens, 2) )

  df['bleu_sm'] = bleu_sm
  df['met'] = met
  df['nist'] = nist

In [14]:
############ Evaluate various models on various feature selections

col_set = []
col_set.append(['bleu_uni'])
col_set.append(['bleu_sm'])
col_set.append(['met'])
col_set.append(['nist'])
col_set.append(['bleu_uni','bleu_sm','met','nist'])
col_set.append(['cos_s_r'])
col_set.append(['cos_s_c'])
col_set.append(['cos_c_r'])
col_set.append(['cos_s_r','cos_s_c','cos_c_r'])
col_set.append(['bleu_uni','cos_s_r','cos_s_c','cos_c_r'])
col_set.append(['bleu_uni','nist','cos_s_r','cos_s_c','cos_c_r'])
col_set.append(['bleu_uni','bleu_sm','met','nist','cos_s_r','cos_s_c','cos_c_r'])
col_set.append(['bleu_uni','nist','cos_c_r'])

result_data = []

with open("output.csv", "w") as f:

  writer = csv.writer(f)
  writer.writerow(['features','model','f1','accuracy'])

  for col in col_set:

    scaler = StandardScaler()
    
    X_train = build_matrix(col,train_data).astype('float32')
    X_train_scaled = scaler.fit_transform(X_train)

    X_test = build_matrix(col,test_data).astype('float32')
    X_test_scaled = scaler.transform(X_test)

    y_train = np.asarray(train_data['target'])
    y_test = np.asarray(test_data['target'])

    ###### SVC Linear

    model_parameters = {
      'kernel': ['linear'],
        'C': [0.1, 0.5, 1, 5, 10, 50, 100]
    }

    grid_search = GridSearchCV(SVC(), model_parameters, cv=5, scoring="f1")
    grid_search.fit(X_train, y_train)

    y_pred = grid_search.predict(X_test)
    accuracy = 1-(np.mean( np.abs(y_pred - y_test) ) )
    f1 = f1_score(y_test, y_pred)
    result_data.append([(' ').join(col),'SVC linear',f1,accuracy])
    writer.writerow(result_data[-1])

    ###### SVC RBF

    model_parameters = {
      'kernel': ['rbf'],
        'C': [0.1, 0.5, 1, 5, 10, 50, 100],
        'gamma': [0.1, 0.5, 1, 3, 6, 10],
    }

    grid_search = GridSearchCV(SVC(), model_parameters, cv=5, scoring="f1")
    grid_search.fit(X_train, y_train)

    y_pred = grid_search.predict(X_test)
    accuracy = 1-(np.mean( np.abs(y_pred - y_test) ) )
    f1 = f1_score(y_test, y_pred)
    result_data.append([(' ').join(col),'SVC RBF',f1,accuracy])
    writer.writerow(result_data[-1])

    ###### Logistic Regression

    model_parameters = {
        'C': [0.1, 0.5, 1, 5, 10, 50, 100],
        'max_iter': [1000]
    }

    grid_search = GridSearchCV(LogisticRegression(), model_parameters, cv=5, scoring="f1")
    grid_search.fit(X_train, y_train)

    y_pred = grid_search.predict(X_test)
    accuracy = 1-(np.mean( np.abs(y_pred - y_test) ) )
    f1 = f1_score(y_test, y_pred)
    result_data.append([(' ').join(col),'logistic regression',f1,accuracy])
    writer.writerow(result_data[-1])

    ##### KNN

    model_parameters = {
      'algorithm': ['auto'],
        'n_neighbors': range(1, 51),
        'leaf_size': range(5, 61, 5)
    }
    grid_search = GridSearchCV(KNeighborsClassifier(), model_parameters, cv=5, scoring="f1")
    grid_search.fit(X_train, y_train)

    y_pred = grid_search.predict(X_test)
    accuracy = 1-(np.mean( np.abs(y_pred - y_test) ) )
    f1 = f1_score(y_test, y_pred)
    result_data.append([(' ').join(col),'KNN',f1,accuracy])
    writer.writerow(result_data[-1])

    ##### Decision Tree

    model_parameters = {
        'max_depth': range(1, 51),
        'min_samples_split': range(2, 11)
    }
    grid_search = GridSearchCV(DecisionTreeClassifier(), model_parameters, cv=5, scoring="f1")
    grid_search.fit(X_train, y_train)

    y_pred = grid_search.predict(X_test)
    accuracy = 1-(np.mean( np.abs(y_pred - y_test) ) )
    f1 = f1_score(y_test, y_pred)
    result_data.append([(' ').join(col),'Decision Tree',f1,accuracy])
    writer.writerow(result_data[-1])

    ##### MLP

    model_parameters = {
        'solver': ['lbfgs'], 
        'max_iter': [750,1500], 
        'alpha': 10.0 ** -np.arange(1, 7), 
        'hidden_layer_sizes':np.arange(12, 20), 
        'random_state':[0,4,8]
        }
    
    grid_search = GridSearchCV(neural_network.MLPClassifier(), model_parameters, n_jobs=-1, scoring="f1")
    grid_search.fit(X_train, y_train)

    y_pred = grid_search.predict(X_test)
    accuracy = 1-(np.mean( np.abs(y_pred - y_test) ) )
    f1 = f1_score(y_test, y_pred)
    result_data.append([(' ').join(col),'MLP',f1,accuracy])
    writer.writerow(result_data[-1])

KeyboardInterrupt: ignored

In [ ]:
############ Output summary CSV

results = dict()

for result in result_data:
  features = result[0]
  model = result[1]
  f1 = result[2]
  acc = result[3]

  if features not in results:
    results[features] = dict()
    results[features]['best_f1_model'] = ''
    results[features]['best_f1_score'] = 0.0
    results[features]['average_f1_score'] = 0.0
    results[features]['average_acc_score'] = 0.0
    results[features]['total_seen'] = 0
    results[features]['total_f1_score'] = 0.0
    results[features]['total_acc_score'] = 0.0

  results[features]['total_seen'] += 1
  results[features]['total_f1_score'] += f1
  results[features]['total_acc_score'] += acc
  results[features]['average_f1_score'] = (results[features]['total_f1_score']/results[features]['total_seen'])
  results[features]['average_acc_score'] = (results[features]['total_acc_score']/results[features]['total_seen'])

  if f1 > results[features]['best_f1_score']:
    results[features]['best_f1_score'] = f1
    results[features]['best_f1_model'] = model

with open("output_agg.csv", "w") as f:

  writer = csv.writer(f)

  key1 = list(results.keys())[0]
  col_names = list(results[features].keys())
  col_names.insert(0,'features')
  writer.writerow(col_names[0:5])
  
  for features in results.keys():

    nums = list(results[features].values())
    nums.insert(0,features)
    writer.writerow(nums[0:5])
